In [77]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop

In [ ]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 3
date_of_request= "2022-04-28"
url = "https://seekingalpha.com/api/v3/articles"
    +"?cacheBuster="+date_of_request
    +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"
    +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="
    +nbr_earning_call_metada
    +"&page[number]=1"
url

In [ ]:
# the chrome driver depend on which version of chrome you are running specify the good one
executable_path = "./chromedriver_linux"
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path= executable_path,options=option)
driver.get(url)
site_html = driver.page_source

In [27]:
#a quick loock of the result to check if we get results are get captcha/bot error 
site_html[0:300]

'<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">{"data":[{"id":"4504314","type":"transcript","attributes":{"publishOn":"2022-04-27T21:00:05-04:00","isLockedPro":false,"commentCount":0,"gettyImageUrl":null,"themes":{},"title":"Informatica Inc. (INFA) CEO Amit Walia'

In [71]:
api_json_response = BeautifulSoup(site_html)
api_data_response = json.loads(api_json_response.get_text())
api_data_response

{'data': [{'id': '4504314',
   'type': 'transcript',
   'attributes': {'publishOn': '2022-04-27T21:00:05-04:00',
    'isLockedPro': False,
    'commentCount': 0,
    'gettyImageUrl': None,
    'themes': {},
    'title': 'Informatica Inc. (INFA) CEO Amit Walia on Q1 2022 Results - Earnings Call Transcript'},
   'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
    'sentiments': {'data': []},
    'primaryTickers': {'data': [{'id': '613018', 'type': 'tag'}]},
    'secondaryTickers': {'data': []},
    'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
      {'id': '17994', 'type': 'tag'},
      {'id': '49', 'type': 'tag'},
      {'id': '586376', 'type': 'tag'},
      {'id': '326', 'type': 'tag'},
      {'id': '17996', 'type': 'tag'}]}},
   'links': {'self': '/article/4504314-informatica-inc-infa-ceo-amit-walia-on-q1-2022-results-earnings-call-transcript'}},
  {'id': '4504313',
   'type': 'transcript',
   'attributes': {'publishOn': '2022-04-27T20:58:05-04:00',
  

In [91]:
siteurl = 'https://seekingalpha.com'
url_earning_calls = [ siteurl + earning_call_metada["links"]["self"] 
                     for earning_call_metada in api_data_response["data"]]
# ""check url
url_earning_calls[0:2]

['https://seekingalpha.com/article/4504314-informatica-inc-infa-ceo-amit-walia-on-q1-2022-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4504313-qualcomm-inc-qcom-ceo-cristiano-amon-on-q2-2022-results-earnings-call-transcript']

In [92]:
# download all html from url
html_transcripts = {} #key is url
for url in tqdm(url_earning_calls):
    html_transcripts[url] = requests.get(url).text

100%|███████████████████████████████████████████| 40/40 [00:08<00:00,  4.57it/s]


In [100]:
html_transcripts.values()

TypeError: 'dict_values' object is not subscriptable

Save html retrieved if not already on disk

In [101]:
import os.path
import pickle

In [104]:
## put in download so we don't download if already have file
html_file_path = "./data/html/"
for url, html_transcript in tqdm(html_transcripts.items()):
    if not os.path.isfile(html_file_path + url):
        file = open(html_file_path+url+".html", "wb")
        pickle.dump(html_transcript, file)

  0%|                                                    | 0/40 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './data/html/https://seekingalpha.com/article/4504314-informatica-inc-infa-ceo-amit-walia-on-q1-2022-results-earnings-call-transcript.html'